In [1]:
import numpy as np
import math
from collections import defaultdict
import queue
import heapq

In [2]:
#Take input from file
file = open ("input.txt" , "r")
line = file.readline()
words = line.strip().split()
n = int (words[0])
node = dict()

#get coordinates of nodes
for x in range(n):
    line = file.readline()
    words = line.strip().split()
    temp , coordinates = int(words[0]) , (int(words[1]) , int (words[2]))
    temp -= 1
    node[temp] = coordinates

words = file.readline().strip().split()
edges = int(words[0])

#get edges and edge weights
graph = [[] for _ in range(n)]
for e in range(edges):
    line = file.readline()
    words = line.strip().split()
    x , y, weight = int (words[0]) - 1 , int (words[1]) - 1 , float(words[2])
    graph[x].append( (y , weight))
    graph[y].append( (x , weight))

#get start and goal state
start = file.readline().strip().split()
start = int(start[0]) - 1

word = file.readline().strip().split()
goal = int(word[0]) - 1

file.close()

In [3]:
print("number of nodes : " , n)
print("Nodes with there coordinates : ")
print(node)

print("Graph : ")
for i in range(n):
    print(i+1, end = " ")
    print(graph[i])


print("Start state : " , start)
print("Goal state : " , goal)

number of nodes :  17
Nodes with there coordinates : 
{0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (0, 3), 4: (1, 0), 5: (1, 1), 6: (2, 0), 7: (3, 1), 8: (3, 2), 9: (3, 3), 10: (4, 1), 11: (5, 0), 12: (5, 1), 13: (5, 2), 14: (5, 3), 15: (6, 1), 16: (6, 2)}
Graph : 
1 [(4, 1.0), (1, 2.5)]
2 [(0, 2.5), (2, 2.5)]
3 [(1, 2.5), (8, 6.0), (3, 3.0), (9, 2.5)]
4 [(2, 3.0), (9, 3.0)]
5 [(0, 1.0), (5, 6.0)]
6 [(4, 6.0), (6, 4.0), (7, 6.0), (8, 10.0)]
7 [(5, 4.0), (11, 10.0)]
8 [(5, 6.0), (10, 6.0)]
9 [(2, 6.0), (5, 10.0), (13, 15.0), (14, 20.0)]
10 [(2, 2.5), (3, 3.0), (14, 10.0)]
11 [(7, 6.0), (12, 4.0)]
12 [(6, 10.0), (12, 2.0)]
13 [(10, 4.0), (11, 2.0), (13, 12.0), (15, 11.0)]
14 [(8, 15.0), (12, 12.0), (14, 5.0), (16, 2.5)]
15 [(8, 20.0), (9, 10.0), (13, 5.0)]
16 [(12, 11.0), (16, 1.0)]
17 [(13, 2.5), (15, 1.0)]
Start state :  0
Goal state :  16


In [4]:
#Euclidean Heuristic
def heuristic(x, speed):
    return math.sqrt((node[goal][0] - x[0])**2 + (node[goal][1] - x[1])**2) / speed

In [5]:
#solve using a star algo
#congestion is sent as a percentage
def astar(congestion):
    cycle = 25.0
    #congestion linearly reduses bus spead to 10
    bus = 10.0 + 40.0 * (100 - congestion)/100 
    speed = max(cycle, bus)
    #store the state of each node as parent, g(n), h(n)
    state = dict()
    visited = set()
    Q = []
    heapq.heapify(Q)
    
    for i in range(n):
        state[i] = (-1, math.inf, heuristic(node[i], speed))
    state[start] = (-1, 0, heuristic(node[start], speed))
    heapq.heappush(Q, (state[start][1] + state[start][2], start))
    
    counter = 0
    while Q:
        counter += 1
        d, curr = heapq.heappop(Q)
        
        if(curr == goal):
            printPath(state)
            return True
        
        visited.add(curr)
        
        for v in graph[curr]:
            if(v[0] not in visited):
                if(v[1] <= 3): #bus cannot go
                    if(state[v[0]][1] > state[curr][1] + v[1]/cycle):
                        state[v[0]] = (curr, state[curr][1] + v[1]/cycle, heuristic(node[v[0]], speed))            
                        heapq.heappush(Q, (state[v[0]][1] + state[v[0]][2], v[0]))
                else: #both can go, we go by fastest (no budget limitation)
                    if(state[v[0]][1] > state[curr][1] + v[1]/speed):
                        state[v[0]] = (curr, state[curr][1] + v[1]/speed, heuristic(node[v[0]], speed))
                        heapq.heappush(Q, (state[v[0]][1] + state[v[0]][2], v[0]))
                    
            if(v[0] not in visited and state[v[0]][1] > state[curr][1] + v[1]):
                state[v[0]] = (curr, state[curr][1] + v[1], heuristic(node[v[0]]))
                heapq.heappush(Q, (state[v[0]][1] + state[v[0]][2], v[0]))
    
    return False

In [6]:
def printPath(state):
    current = goal
    path = []
    while(current != -1):
        path.insert(0, current)
        current = state[current][0]
    print("Path:", end = " ")
    for i in path:
        print(i + 1, end = " ")
    print()
    print("Total Time: " + str(state[goal][1] * 60) + " minutes")

In [7]:
P = astar(100)
if(not P):
    print("Path not found")

Path: 1 2 3 10 15 14 17 
Total Time: 60.000000000000014 minutes
